# COMP 551 final project code
paper: Unleashing the Potential of CNNs for Interpretable Few-Shot Learning

## data loading 
meta training set: 64 classes - training CNN
meta testing set: 20 classes - extract VC and validate
meta validation set: 16 classes - Not being used

On each trial (5-shot)
- Use 25 samples(5 classes) from training split to train CNN to achieve high accuracy.
- Use 25 samples(5 classes) from train subset of the testing split to extract VC
- Use 15 samples for each class(5 classes) form test subset of the testing split to classify images using VC.

10 trials:

desired output file: 

training set : (64 classes in total)
- train_x_1.csv : 5 sample each of 5 classes , train_y_1.csv : 5 sample for each of 5 classes
- test_x_1.csv : 5 sample each of 3 classes, test_y_1.csv : 5 sample each of 3 classes

- train_x_2.csv.....
- test_x_2.csv
....
....
- train_x_10.csv.....
- test_x_10.csv

testing set : (20 classes in total)
- train_x_1.csv : 5 sample each of 5 classes , train_y_1.csv : 5 sample for each of 5 classes
- test_x_1.csv : 5 sample each of 3 classes, test_y_1.csv : 5 sample each of 3 classes

- train_x_2.csv.....
- test_x_2.csv
....
....
- train_x_10.csv.....
- test_x_10.csv

the mini-imagenet structure
```
paper_reproduce_dataset/train/n01882714/images/n01882714_0.jpeg
```

use load_labels() to controll the number of class uses in a trial
then use load_data to get train_x, train_y



In [1]:
#helper function to import data
import re

def tryint(s):
    try:
        return int(s)
    except:
        return s

def alphanum_key(s):
    """ Turn a string into a list of string and number chunks.
        "z23a" -> ["z", 23, "a"]
    """
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]

def sort_nicely(l):
    """ Sort the given list in the way that humans expect.
    """
    l.sort(key=alphanum_key)
    return l

#load data
import skimage
from skimage.io import ImageCollection,concatenate_images,imread

from skimage.color import gray2rgb
import numpy as np
from skimage import io
from os import listdir
from os.path import isfile, join


def print_image(data):
    from matplotlib import pyplot as plt
    plt.imshow(data, interpolation='nearest')
    plt.show()

def load_labels():
    labels = []
    with open("paper_reproduce_dataset/wnids.txt","r") as input:
        for row in input:
            labels.append(row[0:9])
    return labels

def imreadconvert(Xname):
    
    X=imread(Xname)
    if len(X.shape)==3:
        return X
    else:
        return gray2rgb(X)  
    
def load_data(label_dict,dataset):
    print ("loading",dataset, "data!")
    nsamples=10 #5 images for each of 200 labels
    file_names=[]
    labels=[] 
    
    for label in label_dict:
        #print str(label) 
        cur_dir="paper_reproduce_dataset/"+label+"/images"
        onlyfiles = [f for f in sort_nicely(listdir(cur_dir)) if isfile(join(cur_dir, f))]
        onlyfiles = random.sample(onlyfiles,nsamples)
        onlyfiles=[cur_dir+'/'+f for f in onlyfiles]
        file_names=file_names+onlyfiles    
        cur_labels=nsamples*[label]
        labels=labels+cur_labels
    image_collect = ImageCollection(file_names,load_func=imreadconvert)
    x_data = concatenate_images(image_collect)   
    print ("loaded",dataset, "data")
    y_data=np.asarray(labels)
    y_data=np.reshape(y_data,(len(y_data),))
   
    print("x_",dataset,".shape =",x_data.shape)
    print("y_",dataset,".shape =",y_data.shape)
   
    return x_data,y_data 
        

    

In [2]:
#VGG model structure
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D , AveragePooling2D,ZeroPadding2D


def VGG_13(num_class):
    model = Sequential()
    
    
    model.add(Conv2D(64, (3, 3), activation="relu",padding="same",input_shape=(64,64,3)))
    model.add(Conv2D(64, (3, 3), activation="relu",padding="same"))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(Conv2D(128, (3, 3), activation="relu",padding="same"))
    model.add(Conv2D(128, (3, 3), activation="relu",padding="same"))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Conv2D(256, (3, 3), activation="relu",padding="same"))
    model.add(Conv2D(256, (3, 3), activation="relu",padding="same"))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(Conv2D(512, (3, 3), activation="relu",padding="same"))
    model.add(Conv2D(512, (3, 3), activation="relu",padding="same"))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(Conv2D(512, (3, 3), activation="relu",padding="same"))
    model.add(Conv2D(512, (3, 3), activation="relu",padding="same"))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_class, activation='softmax'))

#     if weights_path:
#         model.load_weights(weights_path)

    return model



Using TensorFlow backend.


In [3]:
#data split and train model function
import random

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer

from keras import optimizers

def split_metadata():
    label_dict=load_labels()
    train_split_label = label_dict[0:64]
    test_split_label = label_dict[64:80]
    
    #spliting dataset to 64 training class with 10 samples per class
    #and 16 testing class with 10 samples per class
    x_train,y_train = load_data(train_split_label,"train")
    print()
    x_test,y_test = load_data(test_split_label,"test")
    return x_train,y_train,x_test,y_test

def run_CNN_trial(train_trial):
    #set hyper param
    epochs = 30
    learning_rate = 0.01
    
    train_trial = list(zip(*train_trial))
    x_tr = np.asarray(train_trial[0])
    y_tr = np.asarray(train_trial[1])
    
    #preprocessing training data
    lb = preprocessing.LabelBinarizer()
    onehot_y_tr = lb.fit_transform(y_tr)
    num_class = onehot_y_tr.shape[1]
    x_train_tr,x_test_tr,y_train_tr,y_test_tr = train_test_split(x_tr, onehot_y_tr, test_size=0.5, random_state=42)
    
    cnn_model = VGG_13(num_class)
    print("start training CNN model.")
    
    optimizer = optimizers.SGD(lr=learning_rate)
    cnn_model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=optimizer,
                  metrics=['accuracy'])

    cnn_model.fit(x_train_tr, y_train_tr,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test_tr, y_test_tr))

    score = cnn_model.evaluate(x_test_tr, y_test_tr, verbose=0)
    
#     print('Test loss:', score[0])
#     print('Test accuracy:', score[1])
    
#     print("finish training CNN model.")
    
    return cnn_model

def train_model():
    
    x_train,y_train,x_test,y_test = split_metadata()
   
    train_data = list(zip(x_train,y_train))
    test_data = list(zip(x_test,y_test))
    #random choose 50 samples from training split to train CNN for one trail
    train_trial = random.sample(train_data,50)
    
#     cur_image = "paper_reproduce_dataset/"+"n01443537/images/"+"n01443537_0.JPEG"
#     print_image(imreadconvert(cur_image))
    
#     print_image(train_trial[0][0])
#     print(train_trial[0][1])
#     print_image(train_trial[1][0])
#     print(train_trial[1][1])
#     print_image(train_trial[2][0])
#     print(train_trial[2][1])
    
    cnn_model = run_CNN_trial(train_trial)
    return x_test,y_test,cnn_model 

from keras import backend as K

#get intermidiate layer output from model
def get_int_layer(model,input):
    get_3rd_layer_output = K.function([model.layers[0].input],
                                      [model.layers[8].output])
    layer_output = get_3rd_layer_output([input])[0]
    return layer_output

In [4]:
#train model
x_test,y_test,cnn_model = train_model()   

loading train data!
loaded train data
x_ train .shape = (640, 64, 64, 3)
y_ train .shape = (640,)

loading test data!
loaded test data
x_ test .shape = (160, 64, 64, 3)
y_ test .shape = (160,)
start training CNN model.
Train on 25 samples, validate on 25 samples
Epoch 1/30
25/25 [==============================] - 4s - loss: 4.3266 - acc: 0.0800 - val_loss: 15.4734 - val_acc: 0.0400
Epoch 2/30
25/25 [==============================] - 2s - loss: 14.1839 - acc: 0.1200 - val_loss: 15.4734 - val_acc: 0.0400
Epoch 3/30
25/25 [==============================] - 2s - loss: 15.4734 - acc: 0.0400 - val_loss: 15.4734 - val_acc: 0.0400
Epoch 4/30
25/25 [==============================] - 2s - loss: 16.1181 - acc: 0.0000e+00 - val_loss: 15.4734 - val_acc: 0.0400
Epoch 5/30
25/25 [==============================] - 2s - loss: 12.8945 - acc: 0.2000 - val_loss: 15.4734 - val_acc: 0.0400
Epoch 6/30
25/25 [==============================] - 2s - loss: 12.8945 - acc: 0.2000 - val_loss: 15.4734 - val_acc: 0.0

In [7]:
layer_output = get_int_layer(cnn_model,x_test)
print(x_test.shape)
print(layer_output.shape)
cnn_model.summary()

(160, 64, 64, 3)
(160, 8, 8, 256)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 64)        1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16

In [10]:
# K-mean clustering visual concept
from sklearn.cluster import KMeans

num_feature = layer_output.shape[1] * layer_output.shape[2] * layer_output.shape[3]
input_kmean = layer_output.reshape(160,num_feature)
input_kmean[input_kmean == np.inf] = 0
input_kmean[input_kmean == -np.inf] = 0
input_kmean
kmeans = KMeans(random_state=0).fit(input_kmean)
print(kmeans.labels_)
visual_concepts = kmeans.cluster_centers_
print(visual_concepts)

[7 2 6 1 0 0 7 4 5 1 3 0 0 3 7 0 5 6 3 3 0 0 0 5 0 3 0 5 0 0 1 0 6 0 3 6 4
 5 0 2 2 1 0 0 0 0 1 1 0 0 2 0 7 2 0 4 6 3 4 2 3 1 2 0 3 2 1 0 4 1 4 7 0 0
 7 2 0 3 6 0 4 2 6 7 6 6 0 7 3 0 7 7 0 4 7 7 0 4 0 6 6 0 1 3 1 3 3 4 3 3 1
 3 0 1 0 4 4 1 1 3 7 2 0 2 6 3 1 0 2 0 2 6 0 7 5 3 4 6 0 6 3 3 0 4 1 1 2 1
 7 6 6 4 1 1 2 4 2 0 4 0]
[[  5373268.            0.     12979859.    ...,   2008464.75    1936043.875
    2130552.5  ]
 [  6115673.5           0.     14447947.    ...,   1549097.75    1506042.875
    1671714.375]
 [  3120019.25          0.      7645023.    ...,   1406029.25    1404906.5
    1260878.   ]
 ..., 
 [  8974599.            0.     21627882.    ...,   2866364.5     2699719.
    3200104.75 ]
 [  5984375.5           0.     14401364.    ...,   2486323.25    2468401.5
    2645972.5  ]
 [  3183753.75          0.      7673630.5   ...,   1928382.25    1891293.875
    2085154.   ]]


In [ ]:
#VC-encoding

#Nearest Neighbor